# 영화 리뷰데이터를 이용한 정서분류

## 1. 가설
 - 나이브 베이즈 분류기를 사용하여 영화 리뷰데이터를 긍정 부정 감정으로 분류할 수 있다. 

## 2. 연구방법
 
### 2.1 데이터 수집
 - 데이터 출처 : 네이버 영화 평가 댓글 https://github.com/e9t/nsmc/
 - This is a movie review dataset in the Korean language. Reviews were scraped from Naver Movies.
 - The dataset construction is based on the method noted in Large movie review dataset from Maas et al., 2011
 - Each file is consisted of three columns:  id ,  document ,  label  ◦ id : The review id, provieded by Naver
     > - id : The review id, provieded by Naver 
     > - document : The actual review
     > - label : The sentiment class of the review. (0: negative, 1: positive)
     > - Columns are delimited with tabs (i.e.,  .tsv  format; but the file extension is  .txt  for easy access for novices)
 - Quick peek
<img src= 'dataSample.PNG' width=600 height=500>

### 2.2 데이터 읽기
 - 텍스트 파일에서 헤더를 제거하고 document 부분과 label 부분을 분리하여 리스트로 저장

In [47]:
def loadDataSet(file):
    with open(file,'r',encoding="utf8") as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] #header 제외 : 텍스트 맨 위 id, document, label 
        dataSet = [inst[1] for inst in data] # document 
        labels = [int(inst[2]) for inst in data] # label
    return dataSet, labels

train_dataSet, train_labels = loadDataSet('ratings_test.txt')

In [3]:
print(len(train_dataSet))
print(len(train_labels))

50000
50000


### 2.3 형태소 분리
 - 각 document를 형태소 단위로 tokenizing하기
 - KoNLPy API 사용
 - 참고 사이트 : http://konlpy-ko.readthedocs.org/ko/latest/api/konlpy.tag/#module-konlpy.tag._twitter
 - Twitter Class의 nouns 함수 사용

In [48]:
from konlpy.tag import Twitter
tagger = Twitter()
def tokenize(doc):
    return tagger.nouns(doc)

train_doc = [tokenize(row) for row in train_dataSet]

In [49]:
# 형태소 tokenize 확인
print(train_doc)

[[], [], ['뭐', '이', '평점', '점', '리', '더', '더욱'], ['완전', '막장', '임', '돈', '보기'], ['만', '별', '다섯', '개', '왜', '로', '제', '심기'], ['음악', '주가', '최고', '음악', '영화'], ['진정', '쓰레기'], ['마치', '미국', '애니', '창의력', '로봇', '디자인', '고개'], ['갈수록', '개판', '중국영화', '유치', '내용', '폼', '말', '무기', '유치', '남무', '동사서독', '영화', '이건', '류', '류작'], ['이별', '아픔', '뒤', '인연', '기쁨', '모든', '사람'], ['오랜만', '포켓몬스터', '잼밌', '어요'], ['한국', '독립영화', '한계', '아버지', '비교'], ['청춘', '그', '움', '이성', '찰나', '움', '포착', '섬세', '수채화', '퀴어', '영화'], ['눈', '반전', '영화', '흡인'], ['스토리', '연출', '연기', '비주', '얼', '등', '영화', '기본', '조차', '영화', '무슨', '평', '해', '영화', '김문옥', '감독', '내', '영화', '경력', '몇', '조무래기', '내', '영화', '평론', '마인드'], ['소위', '평점', '뭐'], ['최고'], ['발연기', '도저히', '진짜', '연기', '상상'], ['나이스'], ['별', '재미', '우려', '챔프', '방송', '몇번'], ['일', '금요일', '나이트메어', '시리즈', '가장', '시리즈', '양산', '헬레', '저', '시리즈', '첫편', '작가', '상상력', '작품', '갈고리', '사지', '고어씬', '지금', '보더', '잔인', '충격'], ['나름', '교훈', '평점', '질', '섹스', '코미디'], ['꽤', '영화'], ['민주화', '시대', '억눌린', '영혼', '관음', '욕구', '분출', '인상

### 2.4 나이브 베이즈 분류기 생성

In [62]:
from numpy import *

# function createVocabList: 유일한 단어 목록 생성
def createVocabList(dataSet):
    vocabSet = set([]) 
    for document in dataSet:
        vocabSet = vocabSet|set(document) 
    return list(vocabSet)

# function setOfWords2Vec: 주어진 문서 내에 어휘 목록에 있는 단어가 존재하는지 아닌지를 확인
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList) 
    for word in inputSet:
        if word in vocabList: 
            returnVec[vocabList.index(word)] = 1 
    return returnVec

# function trainNB0: 나이브 베이스 분류기 훈련
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords); p1Num = ones(numWords)
    p0Denom=2.0; p1Denom=2.0

    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num += (trainMatrix[i])
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += (trainMatrix[i])
            p0Denom += sum(trainMatrix[i])

    p1Vect = p1Num/p1Denom
    p0Vect = p0Num/p0Denom

    return p0Vect, p1Vect, pAbusive

In [63]:
trainMat = []
for postinDoc in train_doc:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

# trainNB0 example
p0V, p1V, pAb = trainNB0(trainMat, train_labels)
print ("\ntrainNB0 example:")
print (p0V)
print (p1V)
print (pAb)


trainNB0 example:
[  2.73470616e-05   2.73470616e-05   1.09388246e-04 ...,   5.46941231e-05
   2.73470616e-05   1.64082369e-04]
[  3.11769291e-05   3.11769291e-05   6.23538581e-05 ...,   3.11769291e-05
   3.11769291e-05   1.24707716e-04]
0.50346


### 2.5 분류기 평가

In [67]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec)
    p0 = sum(vec2Classify * p0Vec)
    if p1>p0:
        return 1
    else:
        return 0

def testingNB():
    listOPosts, listClasses = loadDataSet('ratings_test.txt')
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    # training
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(train_labels))
    # test
    errorCount = 0
    for testIndex in range(0, len(trainMat)):
        thisDoc = array(setOfWords2Vec(myVocabList, listOPosts[testIndex]))    
        if classifyNB(thisDoc,p0V,p1V,pAb) != train_labels[testIndex]:
            errorCount += 1  
    print ('the error rate is:', float(errorCount)/len(listOPosts)*100)
    testEntry = ['별로다','노잼','돈아깝다']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as:', classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['최고','재밌다','감동']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))

In [68]:
testingNB()

the error rate is: 28.518
['별로다', '노잼', '돈아깝다'] classified as: 0
['최고', '재밌다', '감동'] classified as:  0


## 3. 결론 
 - 분류기의 오류율은 28.518%로 확인되었음
 - 분류기의 오류율이 높아서 가설을 입증하기는 어려움
 - 한글 데이터로 정서분류가 쉽지 않음을 인지함 
 - 형태소 분석기만 가지고 명사단위로 분리한 결과 자체가 많이 부정확해서 분류가 잘 되지 않은 요인으로 생각됨
 - 이번 분석에서는 형태소 분석기에 의지하였지만 직접 데이터를 전처리하여 테스트 데이터 집합에 긍정 부정 라벨링을 더 정확하게 해야할 필요성이 있음